In [5]:
import pandas as pd 
import os
from difflib import SequenceMatcher
#import ace_tools as tools

In [6]:
print(os.getcwd())

/Users/arhanjoshi/Desktop/Data Analyst Work/CSV_data_wrangling/Scopus_data/scripts


In [7]:
file_path = r"../data/processed/Unique Affiliation.xlsx"
df = pd.read_excel(file_path)

In [5]:
df.head()

,Affiliation,Count,Percentage
0,"National Renewable Energy Laboratory, Golden, ...",1936,0.001968
1,"Naval Research Laboratory, Washington, DC 2037...",1701,0.001729
2,"AT and T Bell Laboratories, Murray Hill, NJ 07...",1550,0.001576
3,"Sandia National Laboratories, Albuquerque, NM ...",1453,0.001477
4,"Bell Laboratories, Murray Hill, NJ 07974, Unit...",1311,0.001333


In [ ]:
df['Affiliation'].str.split(',')

In [ ]:
List(df['Affiliation'].iloc[281502: 281504])                     

In [56]:
list(df['Affiliation'].iloc[281502: 281504])

['Zyvex Instruments, Richardson, TX, 75081, United States, Siemens Energy and Automation, Arlington, TX, United States',
 'Zyvex Instruments, Richardson, TX, 75081, United States']

In [21]:
list(df["Affiliation"][:5])

['National Renewable Energy Laboratory, Golden, 80401, CO, United States',
 'Naval Research Laboratory, Washington, DC 20375, United States',
 'AT and T Bell Laboratories, Murray Hill, NJ 07974, United States',
 'Sandia National Laboratories, Albuquerque, NM 87185, United States',
 'Bell Laboratories, Murray Hill, NJ 07974, United States']

In [16]:
df.shape

(281505, 3)

In [57]:
df.head()


,Affiliation,Count,Percentage
0,"National Renewable Energy Laboratory, Golden, ...",1936,0.001968
1,"Naval Research Laboratory, Washington, DC 2037...",1701,0.001729
2,"AT and T Bell Laboratories, Murray Hill, NJ 07...",1550,0.001576
3,"Sandia National Laboratories, Albuquerque, NM ...",1453,0.001477
4,"Bell Laboratories, Murray Hill, NJ 07974, Unit...",1311,0.001333


In [ ]:
affliation = list(df['Affiliation'].str.split(',')[0])
affliation

['National Renewable Energy Laboratory',
 ' Golden',
 ' 80401',
 ' CO',
 ' United States']

first_parts = df['Affiliation'].apply(lambda s: s.split(',')[0])


In [60]:
df.head()

,Affiliation,Count,Percentage
0,"National Renewable Energy Laboratory, Golden, ...",1936,0.001968
1,"Naval Research Laboratory, Washington, DC 2037...",1701,0.001729
2,"AT and T Bell Laboratories, Murray Hill, NJ 07...",1550,0.001576
3,"Sandia National Laboratories, Albuquerque, NM ...",1453,0.001477
4,"Bell Laboratories, Murray Hill, NJ 07974, Unit...",1311,0.001333


In [70]:
first_parts = df['Affiliation'].apply(lambda s: s.split(',')[0].strip().lower().replace(" ","").replace('&','and'))

In [73]:
first_parts.nunique()

66659

In [8]:
# Sequence Master

# convert excel file to csv file and save
df_aff_excel = pd.read_excel('../data/processed/Unique Affiliation.xlsx')
output_path = '../data/processed/unique_affiliation_uncleaned.csv'
to_csv = df_aff_excel.to_csv(output_path, index=False)


In [9]:
# load csv
df_aff = pd.read_csv('../data/processed/unique_affiliation_uncleaned.csv')
df_aff.head()

,Affiliation,Count,Percentage
0,"National Renewable Energy Laboratory, Golden, ...",1936,0.00196822776136458
1,"Naval Research Laboratory, Washington, DC 2037...",1701,0.00172931581719068
2,"AT and T Bell Laboratories, Murray Hill, NJ 07...",1550,0.00157580218497681
3,"Sandia National Laboratories, Albuquerque, NM ...",1453,0.00147718746759439
4,"Bell Laboratories, Murray Hill, NJ 07974, Unit...",1311,0.0013328236545191


In [11]:
# take sample 1 (0-999)
sample_df = df_aff.iloc[:1000].copy()
print(sample_df.head())
print(sample_df.tail())

                                         Affiliation  Count  \
0  National Renewable Energy Laboratory, Golden, ...   1936   
1  Naval Research Laboratory, Washington, DC 2037...   1701   
2  AT and T Bell Laboratories, Murray Hill, NJ 07...   1550   
3  Sandia National Laboratories, Albuquerque, NM ...   1453   
4  Bell Laboratories, Murray Hill, NJ 07974, Unit...   1311   

            Percentage  
0  0.00196822776136458  
1  0.00172931581719068  
2  0.00157580218497681  
3  0.00147718746759439  
4   0.0013328236545191  
                                           Affiliation  Count Percentage
995  Department of Physics and Astronomy, Optical S...     67      <0.1%
996  Department of Physics and Astronomy, Universit...     67      <0.1%
997  Department of Physics and Astronomy, Universit...     67      <0.1%
998  Department of Physics, Carnegie Mellon Univers...     67      <0.1%
999  Department of Physics, Northeastern University...     67      <0.1%


In [10]:
# prepare for clustering deduplication

affliation_list = sample_df['Affiliation'].tolist() #for sample of 1000 data entries.
threshold = 0.90
assigned = set()
clusters = []

NameError: name 'sample_df' is not defined

In [15]:
# main cluster here

for aff in affliation_list:
    if aff in assigned:
        continue
    #start new cluster
    cluster = [aff]
    assigned.add(aff)
    for other in affliation_list:
        if other in assigned:
            continue
        if SequenceMatcher(None, aff, other).ratio() >= threshold:
            cluster.append(other)
            assigned.add(other)
    clusters.append(cluster)

In [49]:
# Build mapping of each original to its canonical form (shortest string)

records = []
for idx, cluster in enumerate(clusters):
    canonical = min(cluster, key=len)
    for member in cluster:
        records.append({
            'Original Affiliation': member,
            'Canonical Affiliation': canonical,
            'Cluster ID': idx
        })

mapping_df = pd.DataFrame(records)

In [50]:
# save to excel
output_path_excel = '../data/output/unique_affiliation_dedupe.csv'
mapping_df.to_csv(output_path_excel, index = False)

In [12]:
# take sample 2 (1,000 - 9,999)
sample_df_2 = df_aff.iloc[1000:10000].copy()
print(sample_df_2.head())
sample_df_2.tail()

                                            Affiliation  Count Percentage
1000  Department of Physics, The Ohio State Universi...     67      <0.1%
1001  Electrical Engineering, Stanford University, S...     67      <0.1%
1002         Intel Corp, Santa Clara, CA, United States     67      <0.1%
1003  KLA-Tencor Corporation, San Jose, CA 95134, 16...     67      <0.1%
1004  Lawrence Berkeley National Laboratory, Berkele...     67      <0.1%


,Affiliation,Count,Percentage
9995,"Louisiana State University, United States",12,<0.1%
9996,"Lucent Technologies, Bell Laboratories Innovat...",12,<0.1%
9997,"Lucent Technologies, Orlando, FL, United States",12,<0.1%
9998,"Magnolia Optical Technologies Inc., 52-B Cummi...",12,<0.1%
9999,"Martin Marietta Electronics Laboratory, Syracu...",12,<0.1%


In [13]:
# prepare for clustering deduplication

affliation_list_2 = sample_df_2['Affiliation'].tolist() #for sample of 10,000 data entries.
threshold = 0.90
assigned = set()
clusters = []

In [14]:
# main cluster here

for aff in affliation_list_2:
    if aff in assigned:
        continue
    #start new cluster
    cluster = [aff]
    assigned.add(aff)
    for other in affliation_list_2:
        if other in assigned:
            continue
        if SequenceMatcher(None, aff, other).ratio() >= threshold:
            cluster.append(other)
            assigned.add(other)
    clusters.append(cluster)

In [15]:
records = []
for idx, cluster in enumerate(clusters):
    canonical = min(cluster, key=len)
    for member in cluster:
        records.append({
            'Original Affiliation': member,
            'Canonical Affiliation': canonical,
            'Cluster ID': idx
        })

mapping_df = pd.DataFrame(records)

In [16]:
# save to excel
output_path_excel = '../data/output/unique_affiliation_dedupe_10k.csv'
mapping_df.to_csv(output_path_excel, index = False)